In [1]:
import pandas as pd

In [2]:
earthquake_counties = pd.read_csv('scrape_earthquake_counties.csv') 
earthquake_counties.head()

,place,county,time,magnitude,latitude,longitude,depth
0,"18km SSE of Lone Pine, CA",Inyo,1593020449240,5.80,36.446833,-117.975167,4.66
1,"17km S of Searles Valley, CA",San Bernardino,1591234331140,5.51,35.614833,-117.428167,8.44
2,"249 km W of Ferndale, California",Humboldt,1589785354591,5.30,40.848300,-127.191800,10.00
3,"251 km W of Ferndale, California",Humboldt,1589784961579,5.00,40.771400,-127.227100,10.00
4,"30km SE of Bodie, CA",Mono,1586615797040,5.24,38.052500,-118.733333,8.45


In [3]:
earthquake_danger = pd.read_csv('earthquake_danger_csv') 
earthquake_danger.head()

,place,county,earthquake_danger_score
0,"18km SSE of Lone Pine, CA",TBD,52.0
1,"17km S of Searles Valley, CA",TBD,52.0
2,"249 km W of Ferndale, California",TBD,52.0
3,"251 km W of Ferndale, California",TBD,52.0
4,"30km SE of Bodie, CA",TBD,52.0


In [4]:
mix = earthquake_counties.join(earthquake_danger, rsuffix="_r")
mix.head()

,place,county,time,magnitude,latitude,longitude,depth,place_r,county_r,earthquake_danger_score
0,"18km SSE of Lone Pine, CA",Inyo,1593020449240,5.80,36.446833,-117.975167,4.66,"18km SSE of Lone Pine, CA",TBD,52.0
1,"17km S of Searles Valley, CA",San Bernardino,1591234331140,5.51,35.614833,-117.428167,8.44,"17km S of Searles Valley, CA",TBD,52.0
2,"249 km W of Ferndale, California",Humboldt,1589785354591,5.30,40.848300,-127.191800,10.00,"249 km W of Ferndale, California",TBD,52.0
3,"251 km W of Ferndale, California",Humboldt,1589784961579,5.00,40.771400,-127.227100,10.00,"251 km W of Ferndale, California",TBD,52.0
4,"30km SE of Bodie, CA",Mono,1586615797040,5.24,38.052500,-118.733333,8.45,"30km SE of Bodie, CA",TBD,52.0


In [5]:
earthquake_danger_score = mix[['place', 'county', 'earthquake_danger_score']]
earthquake_danger_score.head()

,place,county,earthquake_danger_score
0,"18km SSE of Lone Pine, CA",Inyo,52.0
1,"17km S of Searles Valley, CA",San Bernardino,52.0
2,"249 km W of Ferndale, California",Humboldt,52.0
3,"251 km W of Ferndale, California",Humboldt,52.0
4,"30km SE of Bodie, CA",Mono,52.0


In [6]:
eq_counts = earthquake_danger_score.groupby('county').count()
eq_counts = eq_counts.rename(columns={"place":"earthquakes_per_county"}).drop(columns={'earthquake_danger_score'})
eq_counts

,earthquakes_per_county
county,
Humboldt,18
Inyo,3
Lake,1
Mono,1
Orange,1
Plumas,1
San Bernardino,2
San Diego,1
Santa Cruz,1


In [7]:
eq_sums = earthquake_danger_score.groupby('county').sum()
eq_sums

,earthquake_danger_score
county,
Humboldt,922.0
Inyo,156.0
Lake,52.0
Mono,52.0
Orange,52.0
Plumas,52.0
San Bernardino,104.0
San Diego,52.0
Santa Cruz,52.0


In [8]:
fire_counties = pd.read_csv('fire_danger_csv') 
fire_counties = fire_counties.round(0)
fire_counties

,incident_name,county,fire_danger_score
0,Pala Fire,San Diego,87.0
1,Sand Fire,Amador,93.0
2,Sand Fire,El Dorado,100.0
3,Atlas Fire (Southern LNU Complex),Napa,93.0
4,Atlas Fire (Southern LNU Complex),Solano,80.0
...,...,...,...
430,Sanderson Fire,Riverside,60.0
431,Erbes Fire,Ventura,60.0
432,72 Fire,Riverside,67.0
433,Panther Ridge Fire,Santa Cruz,NaN


In [9]:
fire_counts = fire_counties.groupby('county').count()
fire_counts = fire_counts.rename(columns={"incident_name":"fires_per_county"}).drop(columns={'fire_danger_score'})
fire_counts.head()

,fires_per_county
county,
Alameda,11
Amador,5
Butte,14
Calaveras,5
Colusa,7


In [10]:
fire_sums = fire_counties.groupby('county').sum()
fire_sums.head()

,fire_danger_score
county,
Alameda,833.0
Amador,373.0
Butte,1000.0
Calaveras,367.0
Colusa,493.0


In [11]:
disaster_counts = fire_counts.join(eq_counts, how="outer", on="county")
disaster_counts.head()

,fires_per_county,earthquakes_per_county
county,,
Alameda,11,NaN
Amador,5,NaN
Butte,14,NaN
Calaveras,5,NaN
Colusa,7,NaN


In [12]:
disaster_sums = fire_sums.join(eq_sums, how="outer", on="county")
disaster_sums.head()

,fire_danger_score,earthquake_danger_score
county,,
Alameda,833.0,NaN
Amador,373.0,NaN
Butte,1000.0,NaN
Calaveras,367.0,NaN
Colusa,493.0,NaN


In [13]:
all_scores = disaster_counts.join(disaster_sums, how="outer", on="county").fillna(1).reset_index()
all_scores.head()

,county,fires_per_county,earthquakes_per_county,fire_danger_score,earthquake_danger_score
0,Alameda,11,1.0,833.0,1.0
1,Amador,5,1.0,373.0,1.0
2,Butte,14,1.0,1000.0,1.0
3,Calaveras,5,1.0,367.0,1.0
4,Colusa,7,1.0,493.0,1.0


In [14]:
i=0
for index, row in all_scores.iterrows():
    fire_freq = round(row[1]/7,2)
    eq_freq = round(row[2]/7,2)
    fire_danger= round(row[3]/7,2)
    eq_danger = round(row[4]/7,2)
    all_scores.at[i,'fire_freq'] = fire_freq
    all_scores.at[i,'fire_danger'] = fire_danger
    all_scores.at[i,'eq_freq'] = eq_freq
    all_scores.at[i,'eq_danger'] = eq_danger
    i=i+1
all_scores.head()

,county,fires_per_county,earthquakes_per_county,fire_danger_score,earthquake_danger_score,fire_freq,fire_danger,eq_freq,eq_danger
0,Alameda,11,1.0,833.0,1.0,1.57,119.00,0.14,0.14
1,Amador,5,1.0,373.0,1.0,0.71,53.29,0.14,0.14
2,Butte,14,1.0,1000.0,1.0,2.00,142.86,0.14,0.14
3,Calaveras,5,1.0,367.0,1.0,0.71,52.43,0.14,0.14
4,Colusa,7,1.0,493.0,1.0,1.00,70.43,0.14,0.14


In [15]:
i=0
for index, row in all_scores.iterrows():
    fire_danger = row[5]*row[6]
    all_scores.at[i,'fire_danger'] = round(fire_danger)
    eq_danger = row[5]*row[6]
    all_scores.at[i,'earthquake_danger'] = round(eq_danger)
    total_danger = (fire_danger+eq_danger)/2
    all_scores.at[i,'total_danger'] = total_danger
    i=i+1
all_scores.sort_values('total_danger', ascending=False).head()

,county,fires_per_county,earthquakes_per_county,fire_danger_score,earthquake_danger_score,fire_freq,fire_danger,eq_freq,eq_danger,earthquake_danger,total_danger
31,Riverside,43,1.0,3152.0,1.0,6.14,2765.0,0.14,0.14,2765.0,2764.7806
15,Lassen,20,1.0,1394.0,1.0,2.86,570.0,0.14,0.14,570.0,569.5404
37,San Luis Obispo,19,1.0,1400.0,1.0,2.71,542.0,0.14,0.14,542.0,542.0000
16,Los Angeles,18,1.0,1289.0,1.0,2.57,473.0,0.14,0.14,473.0,473.2398
35,San Diego,16,1.0,1119.0,52.0,2.29,366.0,0.14,7.43,366.0,366.0794


In [16]:
final_scores = all_scores[['county', 'total_danger']]

final_scores.to_csv('county_danger_scores.csv', index=False)
final = pd.read_csv('county_danger_scores.csv') 
final.head()

,county,total_danger
0,Alameda,186.8300
1,Amador,37.8359
2,Butte,285.7200
3,Calaveras,37.2253
4,Colusa,70.4300
